## This file processes the output and coverts to JSON.

In [ ]:
import pandas as ps
from pandas import DataFrame 
import json
import re
import ast

In [3]:
output = ps.read_csv("./rag.csv")

In [6]:
data = []
for index, row in output.iterrows():
    word = row['word']
    genre = row['genre']
    try:
        question = row['formatted_text'].strip()
        
        # if word not included, ignore the response
        if question.find(" ________________ ") == -1:
            continue

        #print("Before: {}".format(question))
        question = re.sub(r'\r?\n',' ',question)
        question = re.sub(r' +',' ',question).strip()

        # format question
        if question.find("The paragraph should be") != -1:
            sp = question.split('.')
            sp.pop(0)
            question = ".".join(sp)
        
        if question.find("Here is the paragraph:") != -1:
            sp = question.split(':')
            sp.pop(0)
            question = ":".join(sp)

        choices_with_def = row['answer_choices']
        choices_with_def_array = ast.literal_eval(choices_with_def)

        answer_choices = []
        for key in  choices_with_def_array:
            answer_choices.append(key)

        row_output = {
            "word": word,
            "genre": genre, 
            "semantic_score": row['semantic_score'],
            "prompt": row['prompt'], 
            "question": question, 
            "ans_choices": answer_choices,
            "ans_choices_with_def": choices_with_def_array,
            "validation_output": row['validation_output'], 
            "transition_score":row['transition_score'], 
            "is_valid":row['is_valid']
        }
        
        data.append(json.dumps(row_output))
    except Exception as err:
        print(Exception, err)
 

In [7]:
with open("questions.jsonl", "w") as f:
    for i in range(len(data)):
        f.write(data[i])
        f.write('\n')

In [8]:
print(data[0])

{"word": "impasse", "genre": "korean war", "semantic_score": 0.560485957, "prompt": "Given the meaning of the word impasse: a situation in which a conflict or problem cannot be resolved because of a lack of agreement or a stale; Generate a paragraph on korean war with the word impasse in it. Make sure your paragraph is one single paragraph that is formally worded. If you are done generating the paragraph, stop. Make sure to use the given word impasse as is and only use it once.", "question": "The Korean War, which lasted from 1950 to 1953, was a stalemate that reached an ________________ in the early 1950s. The war began when North Korean forces, backed by China and the Soviet Union, crossed the 38th parallel and invaded South Korea, prompting a response from the United Nations and the United States.", "ans_choices": ["breakthrough", "blanch", "sycophant", "impasse"], "ans_choices_with_def": {"breakthrough": "a sudden and significant improvement or discovery in a particular area of stu